In [41]:
import pandas as pd
import duckdb as ddb
import json

In [ ]:
with open('response.json') as f:
    data = json.load(f)

In [43]:
location_df = pd.DataFrame([data['location']])
weather_df = pd.DataFrame([data['current']])
weather_df

,last_updated_epoch,last_updated,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,...,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,1732945500,2024-11-30 11:15,24.2,75.6,1,"{'text': 'Mist', 'icon': '//cdn.weatherapi.com...",15.2,24.5,10,N,...,70.3,24.3,75.7,18.2,64.7,4.0,2.0,2.9,19.4,31.2


In [44]:
flattened_data = {**data["location"], **data["current"]}

# Convert to a DataFrame
df = pd.DataFrame([flattened_data])
df

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime,last_updated_epoch,last_updated,...,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,Bengaluru,Karnataka,India,12.9833,77.5833,Asia/Kolkata,1732946328,2024-11-30 11:28,1732945500,2024-11-30 11:15,...,70.3,24.3,75.7,18.2,64.7,4.0,2.0,2.9,19.4,31.2


In [45]:
df.columns

Index(['name', 'region', 'country', 'lat', 'lon', 'tz_id', 'localtime_epoch',
       'localtime', 'last_updated_epoch', 'last_updated', 'temp_c', 'temp_f',
       'is_day', 'condition', 'wind_mph', 'wind_kph', 'wind_degree',
       'wind_dir', 'pressure_mb', 'pressure_in', 'precip_mm', 'precip_in',
       'humidity', 'cloud', 'feelslike_c', 'feelslike_f', 'windchill_c',
       'windchill_f', 'heatindex_c', 'heatindex_f', 'dewpoint_c', 'dewpoint_f',
       'vis_km', 'vis_miles', 'uv', 'gust_mph', 'gust_kph'],
      dtype='object')

In [46]:
df = df.drop(columns='condition')

In [47]:
df.columns.value_counts().sum()

np.int64(36)

In [48]:
conn = ddb.connect('../weatherdata')


In [49]:
conn.execute("CREATE SCHEMA IF NOT EXISTS raw")

In [50]:
conn.execute("""
             CREATE TABLE IF NOT EXISTS raw.DAILY_WEATHER_DATA (
                    name VARCHAR(100),
                    region VARCHAR(100),
                    country VARCHAR(100),
                    lat FLOAT,
                    lon FLOAT,
                    tz_id VARCHAR(100),
                    localtime_epoch BIGINT,
                    localtime DATETIME,
                    last_updated_epoch BIGINT,
                    last_updated DATETIME,
                    temp_c FLOAT,
                    temp_f FLOAT,
                    is_day TINYINT,
                    wind_mph FLOAT,
                    wind_kph FLOAT,
                    wind_degree SMALLINT,
                    wind_dir VARCHAR(10),
                    pressure_mb FLOAT,
                    pressure_in FLOAT,
                    precip_mm FLOAT,
                    precip_in FLOAT,
                    humidity TINYINT,
                    cloud TINYINT,
                    feelslike_c FLOAT,
                    feelslike_f FLOAT,
                    windchill_c FLOAT,
                    windchill_f FLOAT,
                    heatindex_c FLOAT,
                    heatindex_f FLOAT,
                    dewpoint_c FLOAT,
                    dewpoint_f FLOAT,
                    vis_km FLOAT,
                    vis_miles FLOAT,
                    uv FLOAT,
                    gust_mph FLOAT,
                    gust_kph FLOAT,
                    PRIMARY KEY (localtime_epoch)
                );

             """)

In [51]:
conn.execute("INSERT INTO raw.DAILY_WEATHER_DATA SELECT * FROM df")

In [52]:
conn.close()